##**Timeline**
- Looked at previous dashboard code to learn what was used and what is needed (libraries and hardware)
- Made an official Formula SAE GitHub account to contain and log changes we made to any code that we used
- Made a mock-up tachometer to experiment on how to improve it and get more experience with working with Arduino
- Experimented with start-up animations for the tachometer
- Modified previous dashboard code to omit code that is no longer needed for the new dashboard (such as: 4 digit 14-segment LED displays, tachometer LED-strip clear functions)
- Experimented with a PWM driver for controlling brightness on the dashboard display and drive a 7-segment RGB 1" display for the gear position indicator
- Tested 2 warning LEDs for engine temperature and engine oil pressure with PWM driver
- Acquired 7-segment RGB 1" display from Adafruit. Soldered 7 wires connected to PWM driver to test functionality
- Acquired 7-segment RGB 1" display with NeoPixel support from RGBDigit. Simplifies amount of pins required and code (only 3 pins and uses NeoPixel library). No need for PWM driver to control 7-segment display.
- Made a mock-up gear position shifter with color matching to tachometer
- Updated dashboard code to support PWM driver and 7-segment display. Rough draft ready for testing
- Acquired main arduino board, CAN BUS shield, and custom power PCB shield for testing new dashboard code. Made prototype of all wiring to test with dynamometer. See **Dyno To Do Procedure**

##**Hardware**
- Arduino Uno
- CAN BUS Shield v2.0
- Custom PCB power shield
- 2 x [Adafruit NeoPixel Stick 8 x 5050 RGB LED](https://www.adafruit.com/product/1426)
  - Libraries: Adafruit_NeoPixel.h library
  - Operational Frequency: 800 kHz datastream
  - Can use any digital output pin
  - 300-500 ohm resistor on data line
  
- [Adafruit 12-chanel 16-bit PWM LED driver SPI](https://www.adafruit.com/product/1455)
  - Libraries: Adafruit_TLC59711.h, SPI.h
  - 9600 baud rate
  - Any digital pin can be used for the SPI data output and SPI clock
  - 12V on V+
  
- [Adafruit 1" 7-segment RGB Display](https://www.adafruit.com/product/1399)
  - Testing only
  
- [RGBDigit 1" 7-segment RGB Display](https://www.rgbdigit.com/shop/index.php?id_product=8&controller=product&id_lang=1)
  - Favored using this version for its simplicity
  - Built in 330 ohm resistor on data pin

##**Code Documentation**
Testing the NeoPixel LED strip

In [ ]:
#include <Adafruit_NeoPixel.h>

#define LED_len 16
#define PIN6 6

Adafruit_NeoPixel strip = Adafruit_NeoPixel(LED_len, PIN6, NEO_GRB + NEO_KHZ800); // initializes pixel strip

uint32_t green = strip.Color(0, 20, 0),
         yellow = strip.Color(20, 20, 0),
         red = strip.Color(20, 0, 0),
         red_max = strip.Color(100, 0, 0),
         blue = strip.Color(0, 0, 20),
         ledArray[16] = {green, green, green, green, green, yellow, yellow, yellow, yellow, yellow, yellow, red, red, red, red, red};

int prev_RPM_Signal;

void setup() {
  // put your setup code here, to run once
  strip.begin();
  strip.setBrightness(100);

  //Startup Animation
  //lr4leds();
  for (int i = 0; i < 3; i ++) {
    lr2mid();
  }
}

void loop() {
  // put your main code here, to run repeatedly:
  // RPM input signal - Map the raw input signal to however 
  int RPM_Signal = map(analogRead(0), 0, 1023, 0, 17);
  
  if ((prev_RPM_Signal != RPM_Signal) or (RPM_Signal == 17)) {
    prev_RPM_Signal = RPM_Signal;
    updateLEDs(RPM_Signal);
  }
  
}

void updateLEDs(int RPM) {
  // Updates LEDs to indicate RPM 
  // Tachometer increases/decreases linearly
  if (RPM == 0) {
    strip.clear();
    strip.show();
  }
  else if (0 < RPM and RPM < 17) {
      // Below max RPM
      for (int i = 0; i < LED_len; i++) {
        if (i < RPM) {
          strip.setPixelColor(i, ledArray[i]);
        }
        else {
          strip.setPixelColor(i, 0);
        }
      }
      strip.show();
    }
    else if (RPM == 17) {
    // At max RPM - Flash red
    // Red on
    for (int i = 0; i < LED_len; i++) {
      strip.setPixelColor(i, red_max);
      }
    strip.show();
    delay(100);
    
    // Red off
    strip.clear();
    strip.show();
    delay(100);
    }
    else {
      // Error indicator - RPM signal is out of range
      strip.clear();
      for (int i = 0; i < LED_len; i = i + 2) {
        strip.setPixelColor(i, blue);
      }
      strip.show();
    }
}

// Start-Up Animations
void lrrl() {
  for (int i = 0; i <= LED_len; i++) {
    strip.setPixelColor(i, ledArray[i]);
    strip.show();
    delay(30);
  }
  
  for (int j = 0; j <= LED_len; j++) {
    strip.setPixelColor(j, 0);
    strip.show();
    delay(30);
  }
}

void lr2mid() {
  for (int i = 0; i <= LED_len/2; i++) {
    strip.setPixelColor(i, blue);
    strip.setPixelColor(LED_len - i, blue);
    strip.show();
    delay(30);
  }

  for (int j = LED_len/2; j > 0; j--) {
    strip.setPixelColor(j-1, 0);
    strip.setPixelColor(LED_len - j, 0);
    strip.show();
    delay(30);
  }
}

void lr4leds() {
  for (int i = 0; i <= LED_len + 4; i++) {
    strip.setPixelColor(i, ledArray[i]);
    strip.setPixelColor(i-4, 0);
    strip.show();
    delay(50);
    
  }
}

###Initializations

This includes the NeoPixel library

In [ ]:
#include <Adafruit_NeoPixel.h>

#define LED_len 16
#define PIN6 6

Adafruit_NeoPixel strip = Adafruit_NeoPixel(LED_len, PIN6, NEO_GRB + NEO_KHZ800); // initializes pixel strip

uint32_t green = strip.Color(0, 20, 0),
         yellow = strip.Color(20, 20, 0),
         red = strip.Color(20, 0, 0),
         red_max = strip.Color(100, 0, 0),
         blue = strip.Color(0, 0, 20),
         ledArray[16] = {green, green, green, green, green, yellow, yellow, yellow, yellow, yellow, yellow, red, red, red, red, red};

int prev_RPM_Signal;